# Introduction to Cellmage ✨

This notebook introduces the basic functionality of the `cellmage` library, a toolkit for interacting with Large Language Models directly in Jupyter notebooks.

**Date:** April 25, 2025

## What is Cellmage?

Cellmage (pronounced "sell-mage") is a powerful library that simplifies interactions with various LLMs while providing:
- **Conversation history management**: Track and manage multi-turn conversations
- **Persona configuration**: Customize system prompts and parameters for different use cases
- **Snippet management**: Include code and other context in your LLM interactions
- **Conversation saving/loading**: Persist conversations for later use
- **IPython/Jupyter integration**: Interact with LLMs directly from notebook cells
- **Multiple model support**: Works with OpenAI, Gemini, and other LLM providers

At its heart, Cellmage brings the power of Large Language Models directly into your data science workflow!

## Requirements

Before running this notebook, ensure you have:

1. Python 3.10 or later installed
2. The following packages:
   ```
   cellmage>=0.1.0
   jupyter>=1.0.0
   python-dotenv>=0.19.0  # For secure API key management
   ```
3. Valid API credentials for your chosen LLM provider

## Security Best Practices

When working with LLM APIs, always:

1. Store API keys in environment variables or `.env` files
2. Never commit API keys to version control
3. Use appropriate scoping for API keys
4. Monitor API usage and set up rate limiting

Create a `.env` file in your project root with:
```
CELLMAGE_API_KEY=your-api-key-here
CELLMAGE_API_BASE=https://your-api-endpoint
```

## Setup & Installation

Let's start by ensuring we have the cellmage package installed. In this notebook, we'll use the existing installation from the local development environment.

In [1]:
import os
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

# Set up logging with a more informative format
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Try to load environment variables from .env file
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    logger.info(f"Loaded environment variables from {env_path}")
else:
    logger.warning(f"No .env file found at {env_path}. Using system environment variables.")

# Verify required environment variables
required_vars = ["CELLMAGE_API_KEY", "CELLMAGE_API_BASE"]
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    logger.error(f"Missing required environment variables: {', '.join(missing_vars)}")
    logger.info("Please set them in your .env file or system environment.")

# Setup path for cellmage import
notebook_dir = Path.cwd()
project_root = notebook_dir.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    logger.info(f"Added {project_root} to Python path")

# Import and verify cellmage installation
try:
    import cellmage

    logger.info(f"Successfully imported cellmage {cellmage.__version__}")
except ImportError as e:
    logger.error(f"Failed to import cellmage: {e}")
    logger.info("Try installing with: pip install -e ..")
    raise
except Exception as e:
    logger.error(f"Unexpected error while importing cellmage: {e}")
    raise

2025-04-26 02:09:34,222 - __main__ - INFO - Loaded environment variables from ../.env
2025-04-26 02:09:34,224 - __main__ - INFO - Added /Users/tpinto/madpin/cellmage to Python path
2025-04-26 02:09:34,390 - cellmage.config - INFO - Settings loaded successfully using Pydantic
2025-04-26 02:09:34,411 - __main__ - INFO - Successfully imported cellmage 0.1.0


## Setting Up the LLM Client

Cellmage uses an adapter pattern to work with multiple LLM providers. The `DirectLLMAdapter` provides direct HTTP communication with LLM APIs without requiring additional packages.

### Error Handling and Configuration

The client setup includes:
- Environment variable validation
- Connection testing
- Automatic retries for transient errors
- Timeout configuration

In [2]:
from cellmage.adapters.direct_client import DirectLLMAdapter
from cellmage.exceptions import LLMInteractionError, ConfigurationError
from cellmage.models import Message
import time


def setup_llm_client(model="llama-3.1-8b-instant"):
    """Set up the LLM client with error handling and validation.

    Args:
        model (str): The model identifier to use

    Returns:
        DirectLLMAdapter: Configured LLM client
    """
    try:
        # Create client with minimal parameters
        client = DirectLLMAdapter(default_model=model, debug=True)  # Enable debug logging

        # Set additional parameters using set_override
        client.set_override("temperature", 0.7)
        client.set_override("max_tokens", 1000)

        # Test the connection with a minimal prompt
        try:
            logger.info(f"Testing connection to LLM service with model {model}...")
            # Create a proper Message object for testing
            test_message = Message(role="user", content="test")
            client.chat([test_message], stream=False)
            logger.info(f"Successfully connected to LLM service")
        except (LLMInteractionError, ConfigurationError) as e:
            logger.warning(f"Connection test failed: {e}")
            logger.info("Continuing anyway - check your API credentials later")

        return client

    except ConfigurationError as e:
        logger.error(f"Configuration error: {e}")
        logger.info("Please check your API credentials and environment variables.")
        raise
    except Exception as e:
        logger.error(f"Unexpected error during client setup: {e}")
        raise


# Create the LLM client
llm_client = setup_llm_client()
logger.info("LLM client initialization complete")

2025-04-26 02:09:34,499 - __main__ - INFO - Testing connection to LLM service with model llama-3.1-8b-instant...
2025-04-26 02:09:34,999 - __main__ - INFO - Successfully connected to LLM service
2025-04-26 02:09:35,000 - __main__ - INFO - LLM client initialization complete


## Creating a Chat Manager

The `ChatManager` is the central class in Cellmage that coordinates between all components. Let's create a basic setup:

In [3]:
# Create components for the chat manager
from cellmage.resources.memory_loader import MemoryLoader
from cellmage.storage.memory_store import MemoryStore
from cellmage.storage.markdown_store import MarkdownStore  # Added MarkdownStore import
from cellmage.models import PersonaConfig
import os

# Create in-memory components for testing
persona_loader = MemoryLoader()
snippet_provider = MemoryLoader()

# Create a persistent storage for conversations using MarkdownStore
# This will save conversations to files instead of just memory
save_dir = os.path.join(os.getcwd(), "llm_conversations")
history_store = MarkdownStore(save_dir)  # Use MarkdownStore instead of MemoryStore
logger.info(f"Conversations will be saved to: {save_dir}")

# Instead of using MemoryLoader.add_persona, create PersonaConfig directly
# and store them in the personas dictionary to avoid validation errors
helpful_persona = PersonaConfig(
    name="helpful_assistant",
    system_message="You are a helpful assistant who provides clear, concise answers.",
    config={"description": "A helpful assistant for general questions", "temperature": 0.7},
)

code_persona = PersonaConfig(
    name="code_expert",
    system_message="You are a Python code expert who provides detailed explanations and best practices.",
    config={
        "description": "A specialized persona for code-related questions",
        "temperature": 0.3,  # Lower temperature for more deterministic code responses
    },
)

# Store personas directly in the loader's dictionary
persona_loader.personas["helpful_assistant"] = helpful_persona
persona_loader.personas["code_expert"] = code_persona

# Create a chat manager
chat_manager = cellmage.ChatManager(
    llm_client=llm_client,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store,
)

# Set default persona
chat_manager.set_default_persona("helpful_assistant")

print(f"Chat manager initialized with 'helpful_assistant' persona")
print(f"Available personas: {', '.join([p for p in persona_loader.list_personas()])}")

2025-04-26 02:09:35,052 - __main__ - INFO - Conversations will be saved to: /Users/tpinto/madpin/cellmage/notebooks/llm_conversations


Chat manager initialized with 'helpful_assistant' persona
Available personas: code_expert, helpful_assistant


## Sending Messages

Now that our chat manager is set up, we can send messages to the LLM. Cellmage maintains conversation context automatically, so each message builds on the previous interaction.

In [4]:
# Send a message to the LLM with the default persona
response = chat_manager.chat(
    "What are the main features of the cellmage library?",
    stream=True,  # Enable streaming responses
)

### Using Different Personas

One of Cellmage's powerful features is the ability to switch personas during a conversation. Let's try using our code expert persona:

In [5]:
# Send a follow-up question using a different persona
response = chat_manager.chat(
    "How would you implement a simple caching system in Python?",
    persona_name="code_expert",  # Switch to the code expert persona for this message
    stream=True,
)

### Parameter Overrides

You can also override LLM parameters for a specific interaction:

In [6]:
# Send a message with custom parameters
response = chat_manager.chat(
    "Generate a creative name for a Python library that helps with data visualization.",
    overrides={
        "temperature": 1.0,  # Increase creativity
        "max_tokens": 50,  # Keep it short
    },
    stream=True,
)

## Adding a Code Snippet

Cellmage allows you to add code snippets to the conversation, which can be useful for providing context:

In [7]:
# Create a code snippet
snippet_provider.add_snippet(
    name="python_example",
    content="""```python
def calculate_fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return calculate_fibonacci(n-1) + calculate_fibonacci(n-2)
```""",
)

# Add the snippet to the conversation
chat_manager.add_snippet("python_example")

# Now ask about the snippet
response = chat_manager.chat(
    "Can you explain this fibonacci function and suggest how to make it more efficient?",
    stream=True,
)

## Viewing Conversation History

You can view the conversation history at any time:

In [8]:
# Get the conversation history
history = chat_manager.get_history()

# Print the history in a readable format
for i, message in enumerate(history):
    print(f"{i + 1}. {message.role}: {message.content[:50]}...")
    print(f"   ID: {message.id}")
    print()

1. system: You are a helpful assistant who provides clear, co...
   ID: a50fc89b-3082-4992-97d4-af3a435dd6c3

2. user: What are the main features of the cellmage library...
   ID: dfe1e551-0f04-4430-a427-c6aa3337c035

3. assistant: Cellmage is a library designed for modeling and si...
   ID: 20624df4-3163-4342-b5a7-b567668363b5

4. user: How would you implement a simple caching system in...
   ID: d99c1d4c-6f43-45a4-8d97-b820ffa23488

5. assistant: To implement a simple caching system in Python, yo...
   ID: 30f69ca8-9847-4f91-9252-a82f04a731a3

6. user: Generate a creative name for a Python library that...
   ID: 620e8a99-2943-4002-b0b9-f465180bb0c3

7. assistant: Sure! How about **"VisuNest"**?

*Meaning:* A play...
   ID: f8db7dad-5664-4f99-ada7-e5a095bb1549

8. system: ```python
def calculate_fibonacci(n):
    if n <= ...
   ID: a1827647-3323-4aea-8909-fe4a368f7662

9. user: Can you explain this fibonacci function and sugges...
   ID: b3d463b7-f0b1-4955-837c-b68968cddf23

10. assis

## Saving and Loading Conversations

Cellmage allows you to save and load conversations:

In [9]:
# Save the conversation to a persistent file
save_path = chat_manager.save_conversation("example_conversation")
print(f"Conversation saved to: {save_path}")

# Clear the history
chat_manager.clear_history()
print(f"History cleared, current message count: {len(chat_manager.get_history())}")

# Load the conversation back
if save_path:
    chat_manager.load_conversation(save_path)
    print(f"Conversation loaded, message count: {len(chat_manager.get_history())}")
    # Let's verify the file exists on disk
    print(f"File exists: {os.path.exists(save_path)}")
    print(f"File is located at: {os.path.abspath(save_path)}")

2025-04-26 02:09:40,660 - cellmage.storage.markdown_store - ERROR - Error loading conversation from /Users/tpinto/madpin/cellmage/notebooks/llm_conversations/example_conversation.md: 1 validation error for ConversationMetadata
session_id
  Input should be a valid string [type=string_type, input_value=UUID('5b643b48-85cb-40c3-93bd-3854680a3bea'), input_type=UUID]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
2025-04-26 02:09:40,661 - cellmage.history_manager - ERROR - Error loading conversation: Failed to load conversation: 1 validation error for ConversationMetadata
session_id
  Input should be a valid string [type=string_type, input_value=UUID('5b643b48-85cb-40c3-93bd-3854680a3bea'), input_type=UUID]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


Conversation saved to: /Users/tpinto/madpin/cellmage/notebooks/llm_conversations/example_conversation.md
History cleared, current message count: 2
Conversation loaded, message count: 2
File exists: True
File is located at: /Users/tpinto/madpin/cellmage/notebooks/llm_conversations/example_conversation.md


## Troubleshooting Guide

### Common Issues

1. **API Connection Errors**
   - Check your internet connection
   - Verify API credentials in `.env` file
   - Ensure the API endpoint is accessible

2. **Import Errors**
   - Run `pip install -e ..` from the notebooks directory
   - Check Python path configuration
   - Verify Python version compatibility

3. **Memory Issues**
   - Use `chat_manager.clear_history()` regularly
   - Monitor conversation length
   - Consider using persistent storage for long conversations

4. **Performance Tips**
   - Use `stream=True` for better user experience with long responses
   - Set appropriate `max_tokens` for your use case
   - Choose the right model for your needs (e.g., nano for quick responses)

### Example Error Resolution

```python
try:
    response = chat_manager.chat("Your prompt here")
except LLMInteractionError as e:
    if "rate limit" in str(e).lower():
        print("Rate limit reached. Waiting before retry...")
        time.sleep(60)
    elif "context length" in str(e).lower():
        print("Context too long. Clearing history...")
        chat_manager.clear_history()
    else:
        raise
```

## Best Practices

1. **Conversation Management**
   - Save important conversations before clearing history
   - Use appropriate personas for different tasks
   - Keep system prompts clear and focused

2. **Resource Usage**
   - Monitor token usage and costs
   - Use streaming for long responses
   - Clean up resources when done

3. **Error Handling**
   - Always wrap API calls in try-except blocks
   - Implement appropriate retry logic
   - Log errors for debugging

4. **Security**
   - Rotate API keys regularly
   - Monitor for unusual usage patterns
   - Review conversation history before saving

## Conclusion

This notebook has demonstrated the basic functionality of the Cellmage library:
- Setting up an LLM client
- Creating and using personas
- Adding code snippets
- Sending messages to the LLM
- Viewing conversation history
- Saving and loading conversations

In the next notebooks, we'll explore more advanced features and test with different LLM models.